# Weather Data from Era5

In [1]:
%pip install cdsapi xarray netcdf4 scipy numpy pandas

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------------------------------ --------- 1.0/1.4 MB 49.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.1 MB ? eta -:--:--
   ----------- ---------------------------- 2.1/7.1 MB 29.6 MB/s eta 0:00:01
   ----------------------- ---------------- 4.2/7.1 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 7.1/7.1 MB 11.0 MB/s eta 0:00:00

   ------ --------------------------------- 1/6 [netcdf4]
   ------------- -------------------------- 2/6 [multiurl]
   -------------------- ------------------- 3/6 [xarray]
   -------------------- ------------------- 3/6 [xarray]
   -------------------- ------------------- 3/6 [xarray]
   -------------------- ------------------- 3/6 [xarray]
   -------------------- ------------------- 3/6 [xarray]
   -------------------- ------------------- 3/6 [xarray]
   ---------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install cfgrib

   ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
   -------------------------- ------------- 4.7/7.2 MB 25.9 MB/s eta 0:00:01
   ---------------------------------------- 7.2/7.2 MB 22.3 MB/s eta 0:00:00

   -------- ------------------------------- 1/5 [pycparser]
   ---------------- ----------------------- 2/5 [cffi]
   ---------------- ----------------------- 2/5 [cffi]
   ------------------------ --------------- 3/5 [eccodes]
   ------------------------ --------------- 3/5 [eccodes]
   -------------------------------- ------- 4/5 [cfgrib]
   -------------------------------- ------- 4/5 [cfgrib]
   -------------------------------- ------- 4/5 [cfgrib]
   ---------------------------------------- 5/5 [cfgrib]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cdsapi
import xarray as xr
import pandas as pd
import numpy as np
import os
import zipfile
import shutil

YEAR = "2024"

# DOWNLOAD ERA5 DATA BY MONTH (ZIP WITH NETCDF FILES)

def download_era5_monthly(
    year=YEAR,
    month="01",
    out_folder="weather_data",
    spatial_resolution=2.5,
    temporal_resolution=6,
    north=40, south=-20,
    east=150, west=60
):
    """
    Downloads ERA5 hourly weather for a given month as ZIP file. Applies spatial and temporal sampling.
    """
    
    os.makedirs(out_folder, exist_ok=True)
    zip_file = os.path.join(out_folder, f"era5_{year}_{month}.zip")

    if os.path.exists(zip_file):
        print(f"✔ {zip_file} already exists — skipping download.")
        return zip_file

    c = cdsapi.Client()

    # Generate time list based on temporal resolution
    times = [f"{h:02d}:00" for h in range(0, 24, temporal_resolution)]

    print(f"⬇ Downloading ERA5 data for {year}-{month}...")
    print(f"   Spatial: {spatial_resolution}° grid")
    print(f"   Temporal: Every {temporal_resolution}h ({len(times)} times/day)")

    c.retrieve(
        "reanalysis-era5-single-levels",
        {
            "product_type": "reanalysis",
            "format": "netcdf",
            "variable": [
                "10m_u_component_of_wind",
                "10m_v_component_of_wind",
                "2m_temperature",
                "surface_pressure",
                "mean_sea_level_pressure",
                "total_precipitation",
            ],
            "year": year,
            "month": month,
            "day": [f"{d:02d}" for d in range(1, 32)],
            "time": times,
            "area": [north, west, south, east],
            "grid": [spatial_resolution, spatial_resolution],
        },
        zip_file,
    )

    file_size_mb = os.path.getsize(zip_file) / (1024 * 1024)
    print(f"✔ Downloaded → {zip_file} ({file_size_mb:.1f} MB)")
    return zip_file

# EXTRACT ZIP FILE

def extract_zip(zip_path, extract_folder):
    """Extract zip file to a folder."""
    if not os.path.exists(zip_path):
        print(f"⚠ {zip_path} not found")
        return None
    
    os.makedirs(extract_folder, exist_ok=True)
    
    print(f"📦 Extracting {zip_path}...")
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    
    extracted_files = os.listdir(extract_folder)
    print(f"  Extracted: {extracted_files}")
    
    return extract_folder

# CONVERT NETCDF TO CSV

def convert_monthly_nc_to_csv(extract_folder, month, year=YEAR, output_csv=None):
    """Convert split monthly NetCDF files to CSV."""
    
    if output_csv is None:
        output_csv = f"era5_{year}_{month}.csv"
    
    if os.path.exists(output_csv):
        print(f"✔ {output_csv} already exists — skipping conversion.")
        return output_csv
    
    nc_files = [f for f in os.listdir(extract_folder) if f.endswith('.nc')]
    
    if len(nc_files) == 0:
        print(f"⚠ No NetCDF files in {extract_folder}")
        return None
    
    print(f"📡 Loading {year}-{month}...")
    print(f"  Files: {nc_files}")
    
    datasets = []
    for nc_file in nc_files:
        nc_path = os.path.join(extract_folder, nc_file)
        try:
            ds = xr.open_dataset(nc_path, engine='netcdf4')
            print(f"  ✓ {nc_file}: {list(ds.data_vars)}")
            datasets.append(ds)
        except Exception as e:
            print(f"  ✗ Error loading {nc_file}: {e}")
    
    if len(datasets) == 0:
        return None
    
    print("🔗 Merging datasets...")
    ds_merged = xr.merge(datasets)
    
    print("🛠 Converting to DataFrame...")
    df = ds_merged.to_dataframe().reset_index()
    
    # Check what columns we got
    print(f"  Initial columns: {list(df.columns)}")
    
    # Handle different time column names
    time_col = None
    for col in ['time', 'valid_time', 't']:
        if col in df.columns:
            time_col = col
            break
    
    if time_col and time_col != 'time':
        df = df.rename(columns={time_col: 'time'})
        print(f"  Renamed '{time_col}' to 'time'")
    
    # Compute wind speed
    if 'u10' in df.columns and 'v10' in df.columns:
        df["wind_speed"] = np.sqrt(df["u10"]**2 + df["v10"]**2)
    
    # Select and order columns - INCLUDE TIME
    cols_to_keep = []
    for col in ["time", "latitude", "longitude", "u10", "v10", "wind_speed", "t2m", "sp", "msl", "tp"]:
        if col in df.columns:
            cols_to_keep.append(col)
    
    df_output = df[cols_to_keep] if cols_to_keep else df
    
    print(f"💾 Saving to CSV...")
    df_output.to_csv(output_csv, index=False)
    
    for ds in datasets:
        ds.close()
    
    print(f"✔ {output_csv} ({len(df_output):,} rows)")
    print(f"  Columns: {list(df_output.columns)}")
    return output_csv

# MERGE MONTHLY CSV FILES

def merge_monthly_csvs(year, months, output_csv):
    """
    Merge multiple monthly CSV files into one.
    """
    print("🔗 Merging monthly CSV files...")
    
    csv_files = [f"era5_{year}_{m:02d}.csv" for m in months]
    
    # Check if all files exist
    missing = [f for f in csv_files if not os.path.exists(f)]
    if missing:
        print(f"⚠ Missing files: {missing}")
        return None
    
    print(f"Found {len(csv_files)} CSV files to merge")
    
    # Read and concatenate all CSVs
    dfs = []
    for csv_file in csv_files:
        print(f"  Reading {csv_file}...")
        df = pd.read_csv(csv_file, parse_dates=['time'])
        dfs.append(df)
    
    print("Concatenating dataframes...")
    merged_df = pd.concat(dfs, ignore_index=True)
    
    # Sort by time
    print("Sorting by time...")
    merged_df = merged_df.sort_values('time').reset_index(drop=True)
    
    # Save merged CSV
    print(f"💾 Saving merged CSV...")
    merged_df.to_csv(output_csv, index=False)
    
    print(f"✔ Merged CSV saved → {output_csv}")
    print(f"  Total rows: {len(merged_df):,}")
    
    return output_csv

# MAIN

if __name__ == "__main__":

    OUTPUT_CSV = f"era5_weather_{YEAR}_sampled.csv"
    WEATHER_DATA_FOLDER = "weather_data"

    # SAMPLING CONFIGURATION
    SPATIAL_RES = 2.5   # 2.5° grid (~250km resolution)
    TEMPORAL_RES = 6    # Every 6 hours (00:00, 06:00, 12:00, 18:00)

    # Step A — Download ERA5 weather by month (with sampling)
    print("=" * 60)
    print(f"DOWNLOADING SAMPLED ERA5 DATA FOR {YEAR}")
    print("=" * 60)
    print(f"📊 Spatial: {SPATIAL_RES}° grid (~{int(SPATIAL_RES * 111)}km)")
    print(f"⏰ Temporal: Every {TEMPORAL_RES} hours")
    print("=" * 60)
    
    for m in range(1, 13):
        month = f"{m:02d}"
        
        try:
            download_era5_monthly(
                year=YEAR,
                month=month,
                out_folder=WEATHER_DATA_FOLDER,
                spatial_resolution=SPATIAL_RES,
                temporal_resolution=TEMPORAL_RES,
                north=40,
                south=-20,
                east=150,
                west=60
            )
        except Exception as e:
            print(f"❌ Error downloading {month}: {e}")
            continue

    # Step B — Extract and convert each monthly ZIP to CSV
    print("\n" + "=" * 60)
    print("EXTRACTING AND CONVERTING TO CSV")
    print("=" * 60)
    
    for m in range(1, 13):
        month = f"{m:02d}"
        zip_file = os.path.join(WEATHER_DATA_FOLDER, f"era5_{YEAR}_{month}.zip")
        extract_folder = f"era5_{YEAR}_{month}_extracted"
        csv_file = f"era5_{YEAR}_{month}.csv"
        
        if os.path.exists(zip_file):
            try:
                extract_zip(zip_file, extract_folder)
                convert_monthly_nc_to_csv(extract_folder, month, YEAR, csv_file)
                # Cleanup temporary extraction folder
                shutil.rmtree(extract_folder, ignore_errors=True)
            except Exception as e:
                print(f"❌ Error converting {month}: {e}")
                continue
        else:
            print(f"⚠ Skipping {month} - zip file not found")

    # Step C — Merge all monthly CSVs
    print("\n" + "=" * 60)
    print("MERGING ALL MONTHLY CSV FILES")
    print("=" * 60)
    
    merge_monthly_csvs(
        year=YEAR,
        months=range(1, 13),
        output_csv=OUTPUT_CSV
    )
    
    print("\n✅ All done!")

DOWNLOADING SAMPLED ERA5 DATA FOR 2024
📊 Spatial: 2.5° grid (~277km)
⏰ Temporal: Every 6 hours


2025-12-11 03:13:00,138 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


⬇ Downloading ERA5 data for 2024-01...
   Spatial: 2.5° grid
   Temporal: Every 6h (4 times/day)


2025-12-11 03:13:00,883 INFO Request ID is 36728d35-0312-43a8-bb3b-135b4cf2273c
2025-12-11 03:13:01,078 INFO status has been updated to accepted
2025-12-11 03:13:34,548 INFO status has been updated to running
2025-12-11 03:14:18,015 INFO status has been updated to successful


✔ Downloaded → weather_data\era5_2024_01.zip (1.4 MB)


...

✔ era5_2024_10.csv (114,700 rows)
  Columns: ['time', 'latitude', 'longitude', 'u10', 'v10', 'wind_speed', 't2m', 'sp', 'msl', 'tp']
📦 Extracting weather_data\era5_2024_11.zip...
  Extracted: ['data_stream-oper_stepType-accum.nc', 'data_stream-oper_stepType-instant.nc']
📡 Loading 2024-11...
  Files: ['data_stream-oper_stepType-accum.nc', 'data_stream-oper_stepType-instant.nc']
  ✓ data_stream-oper_stepType-accum.nc: ['tp']
  ✓ data_stream-oper_stepType-instant.nc: ['u10', 'v10', 't2m', 'sp', 'msl']
🔗 Merging datasets...
🛠 Converting to DataFrame...
  Initial columns: ['valid_time', 'latitude', 'longitude', 'tp', 'number', 'expver', 'u10', 'v10', 't2m', 'sp', 'msl']
  Renamed 'valid_time' to 'time'
💾 Saving to CSV...


C:\Users\EE HANN\AppData\Local\Temp\ipykernel_8916\1626240574.py:124: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds_merged = xr.merge(datasets)
C:\Users\EE HANN\AppData\Local\Temp\ipykernel_8916\1626240574.py:124: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds_merged = xr.merge(datasets)


✔ era5_2024_11.csv (111,000 rows)
  Columns: ['time', 'latitude', 'longitude', 'u10', 'v10', 'wind_speed', 't2m', 'sp', 'msl', 'tp']
📦 Extracting weather_data\era5_2024_12.zip...
  Extracted: ['data_stream-oper_stepType-accum.nc', 'data_stream-oper_stepType-instant.nc']
📡 Loading 2024-12...
  Files: ['data_stream-oper_stepType-accum.nc', 'data_stream-oper_stepType-instant.nc']
  ✓ data_stream-oper_stepType-accum.nc: ['tp']
  ✓ data_stream-oper_stepType-instant.nc: ['u10', 'v10', 't2m', 'sp', 'msl']
🔗 Merging datasets...
🛠 Converting to DataFrame...
  Initial columns: ['valid_time', 'latitude', 'longitude', 'tp', 'number', 'expver', 'u10', 'v10', 't2m', 'sp', 'msl']
  Renamed 'valid_time' to 'time'
💾 Saving to CSV...


C:\Users\EE HANN\AppData\Local\Temp\ipykernel_8916\1626240574.py:124: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds_merged = xr.merge(datasets)
C:\Users\EE HANN\AppData\Local\Temp\ipykernel_8916\1626240574.py:124: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds_merged = xr.merge(datasets)


✔ era5_2024_12.csv (114,700 rows)
  Columns: ['time', 'latitude', 'longitude', 'u10', 'v10', 'wind_speed', 't2m', 'sp', 'msl', 'tp']

MERGING ALL MONTHLY CSV FILES
🔗 Merging monthly CSV files...
Found 12 CSV files to merge
  Reading era5_2024_01.csv...
  Reading era5_2024_02.csv...
  Reading era5_2024_03.csv...
  Reading era5_2024_04.csv...
  Reading era5_2024_05.csv...
  Reading era5_2024_06.csv...
  Reading era5_2024_07.csv...
  Reading era5_2024_08.csv...
  Reading era5_2024_09.csv...
  Reading era5_2024_10.csv...
  Reading era5_2024_11.csv...
  Reading era5_2024_12.csv...
Concatenating dataframes...
Sorting by time...
💾 Saving merged CSV...
✔ Merged CSV saved → era5_weather_2024_sampled.csv
  Total rows: 1,354,200

✅ All done!
